In [2]:
import numpy as np
import os
#import h5py
import time
import copy
from random import randint
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4), #crop
    transforms.RandomHorizontalFlip(),   #flip
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size= 4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            #nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            nn.Dropout(p=0.5, inplace=False))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
            
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            #nn.BatchNorm2d(64))
            nn.MaxPool2d(2, 2),
            nn.Dropout(p=0.5, inplace=False))
            
        self.layer5 = nn.Sequential(
            nn.Conv2d(64, 64, 4,stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
            
        self.layer6 = nn.Sequential(
            nn.Conv2d(64, 64, 3,stride=1, padding=0),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False))
            
        self.layer7 = nn.Sequential(
            nn.Conv2d(64, 64, 3,stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64))
            #nn.MaxPool2d(2, 2))
            
        self.layer8 = nn.Sequential(
            nn.Conv2d(64, 64, 3,stride=1, padding=0),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.5, inplace=False))
            
        
        
        self.fc1 = nn.Linear(1024, 120)  
        self.fc2 = nn.Linear(120, 10)
        #self.linear3 = nn.Linear(layer_two_out, 10)
        #self.softmax = nn.LogSoftmax(dim=1)

        
    def forward(self, x):
        #print(x.size())
        out = self.layer1(x)
        #print(out.size())
        out = self.layer2(out)
        #print(out.size())
        out = self.layer3(out)
        #print(out.size())
        #out = out.view(-1, 64 * 18 * 18)
        
        out = self.layer4(out)
        out = self.layer5(out)
        #print(out.size())
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        #print(out.size())
        #out = out.reshape(out.size(0), -1)
        #print(out.size())
        out = out.view(-1, out.shape[1] * out.shape[2] * out.shape[3])
        #print(out.size())
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = out.softmax(1)
        #out = self.linear3(out)
        #out = self.softmax(out)
        
        return out

model = Model().to(device)

class Model1(nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64,4,1,2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 64, 4,1,2)
        self.conv3 = nn.Conv2d(64, 128, 4,1,2)
        self.conv4 = nn.Conv2d(128, 128, 4,1,2)
        self.conv5 = nn.Conv2d(128, 256, 4,1,2)
        self.conv6 = nn.Conv2d(256, 256, 4,1,2)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.drop = nn.Dropout(p = 0.5)
        
        self.fc1 = nn.Linear(256*5*5, 500)
        self.fc2 = nn.Linear(500, 500)
        self.fc3 = nn.Linear(500, 10)

    def forward(self, x):
        #layer1
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        #layer2
        x = F.relu(self.conv2(x))
        x = self.bn1(x)
        x = self.pool(x)
        x = self.drop(x)
        #layer3
        x = F.relu(self.conv3(x))
        x = self.bn2(x)
        #layer4
        
        x = F.relu(self.conv4(x))
        x = self.bn2(x)
        x = self.pool(x)
        x = self.drop(x)
        #layer5
        x = F.relu(self.conv5(x))
        x = self.bn3(x)
        #layer6
        x = F.relu(self.conv6(x))
        x = self.bn3(x)
        x = self.pool(x)
        x = self.drop(x)
        #print(x.size())
        
        x = x.view(-1, 256*5*5)
        x = F.relu(self.fc1(x))
        #x = self.drop(x)
        #x = F.relu(self.fc2(x))
        #x = self.drop(x)
        x = self.fc3(x)
        return x


model1 = Model1().to(device)

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model1.parameters(), lr=0.001)
#scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [4, 6 ,8], gamma=0.1, last_epoch=-1)


In [7]:
#  TRAINING
epoch =80
time1 = time.time()

for epoch in range(epoch):  # loop over the dataset multiple times
    #scheduler.step()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('%d, %5d    loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')
time2 = time.time()
print(time2-time1)

1,  2000    loss: 2.072
1,  4000    loss: 1.830
1,  6000    loss: 1.750
1,  8000    loss: 1.650
1, 10000    loss: 1.589
1, 12000    loss: 1.517
2,  2000    loss: 1.411
2,  4000    loss: 1.357
2,  6000    loss: 1.308
2,  8000    loss: 1.252
2, 10000    loss: 1.224
2, 12000    loss: 1.152
3,  2000    loss: 1.101
3,  4000    loss: 1.088
3,  6000    loss: 1.069
3,  8000    loss: 1.057
3, 10000    loss: 1.023
3, 12000    loss: 1.000
4,  2000    loss: 0.952
4,  4000    loss: 0.952
4,  6000    loss: 0.945
4,  8000    loss: 0.930
4, 10000    loss: 0.919
4, 12000    loss: 0.921
5,  2000    loss: 0.863
5,  4000    loss: 0.922
5,  6000    loss: 0.870
5,  8000    loss: 0.857
5, 10000    loss: 0.862
5, 12000    loss: 0.851
6,  2000    loss: 0.852
6,  4000    loss: 0.807
6,  6000    loss: 0.814
6,  8000    loss: 0.804
6, 10000    loss: 0.784
6, 12000    loss: 0.821
7,  2000    loss: 0.794
7,  4000    loss: 0.779
7,  6000    loss: 0.783
7,  8000    loss: 0.742
7, 10000    loss: 0.776
7, 12000    loss

56,  2000    loss: 0.395
56,  4000    loss: 0.400
56,  6000    loss: 0.421
56,  8000    loss: 0.411
56, 10000    loss: 0.430
56, 12000    loss: 0.438
57,  2000    loss: 0.391
57,  4000    loss: 0.433
57,  6000    loss: 0.398
57,  8000    loss: 0.417
57, 10000    loss: 0.394
57, 12000    loss: 0.417
58,  2000    loss: 0.376
58,  4000    loss: 0.429
58,  6000    loss: 0.408
58,  8000    loss: 0.393
58, 10000    loss: 0.410
58, 12000    loss: 0.410
59,  2000    loss: 0.402
59,  4000    loss: 0.401
59,  6000    loss: 0.408
59,  8000    loss: 0.408
59, 10000    loss: 0.429
59, 12000    loss: 0.395
60,  2000    loss: 0.397
60,  4000    loss: 0.424
60,  6000    loss: 0.393
60,  8000    loss: 0.415
60, 10000    loss: 0.410
60, 12000    loss: 0.421
61,  2000    loss: 0.402
61,  4000    loss: 0.420
61,  6000    loss: 0.383
61,  8000    loss: 0.416
61, 10000    loss: 0.421
61, 12000    loss: 0.400
62,  2000    loss: 0.430
62,  4000    loss: 0.382
62,  6000    loss: 0.412
62,  8000    loss: 0.383


In [8]:
#correct = 0
#total = 0
#with torch.no_grad():
#    for data in testloader:
#        inputs, labels = inputs.to(device), labels.to(device)
#        outputs = model(inputs)
#        _, predicted = torch.max(outputs.data, 1)
#        total += labels.size(0)
#        correct += (predicted == labels).sum().item()

#print('Accuracy of the network on the 10000 test images: %d %%' % (
#    100 * correct / total))

#model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 82.37 %


1,  2000    loss: 1.966
1,  4000    loss: 1.706
1,  6000    loss: 1.602
1,  8000    loss: 1.561
1, 10000    loss: 1.522
1, 12000    loss: 1.470
2,  2000    loss: 1.421
2,  4000    loss: 1.397
2,  6000    loss: 1.377
2,  8000    loss: 1.358
2, 10000    loss: 1.322
2, 12000    loss: 1.318
3,  2000    loss: 1.262
3,  4000    loss: 1.224
3,  6000    loss: 1.232
3,  8000    loss: 1.229
3, 10000    loss: 1.215
3, 12000    loss: 1.227
4,  2000    loss: 1.162
4,  4000    loss: 1.163
4,  6000    loss: 1.138
4,  8000    loss: 1.163
4, 10000    loss: 1.146
4, 12000    loss: 1.109
5,  2000    loss: 1.108
5,  4000    loss: 1.094
5,  6000    loss: 1.105
5,  8000    loss: 1.079
5, 10000    loss: 1.066
5, 12000    loss: 1.063
6,  2000    loss: 1.041
6,  4000    loss: 1.028
6,  6000    loss: 1.042
6,  8000    loss: 1.018
6, 10000    loss: 1.031
6, 12000    loss: 1.036
7,  2000    loss: 1.013
7,  4000    loss: 0.989
7,  6000    loss: 0.981
7,  8000    loss: 1.014
7, 10000    loss: 0.979
7, 12000    loss: 0.994
8,  2000    loss: 0.944
8,  4000    loss: 0.963
8,  6000    loss: 0.948
8,  8000    loss: 0.955
8, 10000    loss: 0.959
8, 12000    loss: 0.984
9,  2000    loss: 0.952
9,  4000    loss: 0.930
9,  6000    loss: 0.926
9,  8000    loss: 0.934
9, 10000    loss: 0.925
9, 12000    loss: 0.948
10,  2000    loss: 0.901
10,  4000    loss: 0.922
10,  6000    loss: 0.940
10,  8000    loss: 0.903
10, 10000    loss: 0.909
10, 12000    loss: 0.920
Finished Training
3275.720374584198

-